In [ ]:
import os
import Image
import numpy
from IPython.display import Image as DisplayImage

In [ ]:
test_path = "../captures/testing/IMG_2114.PNG"
DisplayImage(filename=test_path)

In [ ]:
combinedImage = Image.open(test_path)
combinedImage.size

In [ ]:
def split(image):
    splitHeight = (image.size[1] / 2) - 1
    return image.crop((0, 0, image.size[0], splitHeight)), image.crop((0, splitHeight + 1, image.size[0], image.size[1]))
cameraImage, depthImage = split(combinedImage)

In [ ]:
def convertToDepth(image):
    depths = numpy.zeros(image.size)
    pixels = image.load()
    BYTE_MAX = 255
    CHANNEL_MAX = 8
    MAX_RED_VALUE = BYTE_MAX - CHANNEL_MAX
    CHANNELS_MAX = CHANNEL_MAX * CHANNEL_MAX
    INVALID = float('Nan')
    found_orientation = False
    orientation = [1, 0, 0, 0]
    for y in range(0, image.size[1]):
        for x in range(0, image.size[0]):
            pixel = pixels[x, y]
            red = pixel[0]
            if x == 0 and y == 0 and red == BYTE_MAX:
                found_orientation = True
                depths[x, y] = INVALID
                continue
            if found_orientation:
                for c in range(0, len(orientation)):
                    orientation[c] = ((2.0 * pixel[c]) / BYTE_MAX) - 1
                found_orientation = False
                depths[x, y] = INVALID
                continue
            if red == 0:
                depths[x, y] = INVALID
                continue
            depths[x, y] = (MAX_RED_VALUE - red) * CHANNELS_MAX + ((pixel[1] - red) * CHANNEL_MAX) + (pixel[2] - red)
    return depths, orientation
depths, attitude = convertToDepth(depthImage)

In [ ]:
depths.T

In [ ]:
attitude